In [19]:
!git clone https://github.com/JuanIsernGhosn/coral-classification

fatal: destination path 'coral-classification' already exists and is not an empty directory.


In [0]:
experiment = "coral-classification-deep-learning/experiments/vgg16_Keras_2/"

In [0]:
# Imports
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.framework import ops
from tensorflow.python.framework import dtypes
import numpy
import math
import pandas as pd
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, Flatten, Activation, merge
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, Adam
import keras.backend as K
from natsort import natsorted, ns
import keras
import PIL
from keras.regularizers import l2
from glob import glob
from keras.applications import vgg16
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

In [0]:
# Fijar las semillas
seed = 2032 # Semilla del numpy
tf.set_random_seed(seed)# Fijar semilla del keras/tensorflow

epochs = 1000
batch_size = 32

model_name = 'model.h5'

In [0]:
# Train and test directories
train_dir = "coral-classification/coral_img/train/"
test_dir = "coral-classification/coral_img/Test_Mixed/"

In [24]:
# Classes
clases = sorted(os.listdir(train_dir))
print(clases)

x_train = np.array([cv2.imread(os.path.join(train_dir, cl, name)) for cl in clases
           for name in os.listdir(os.path.join(train_dir, cl))])
y_train = np.array([n for n, cl in enumerate(clases)
           for name in os.listdir(os.path.join(train_dir, cl))])

idx = np.random.permutation(len(x_train))
x_train, y_train = x_train[idx], y_train[idx]

test_files = natsorted(os.listdir(test_dir))

x_test = np.array([cv2.imread(os.path.join(test_dir, name)) 
                   for name in test_files])

['ACER', 'APAL', 'CNAT', 'DANT', 'DSTR', 'GORG', 'MALC', 'MCAV', 'MMEA', 'MONT', 'PALY', 'SPO', 'SSID', 'TUNI']


In [25]:

x_train = x_train.astype('float32')
x_train /= 255

x_test = x_test.astype('float32')
x_test /= 255

y_train = to_categorical(y_train, dtype=int)

print(x_train.shape)
print(len(clases))
print(y_train.shape)

(620, 256, 256, 3)
14
(620, 14)


In [0]:
datagen = ImageDataGenerator(
    rotation_range=20,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2,
    fill_mode='nearest'
)

In [27]:
# create the base pre-trained model
base_model = vgg16.VGG16(weights='imagenet',include_top=False, input_shape=(256,256,3))

Instructions for updating:
Colocations handled automatically by placer.


In [28]:
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
__________

In [29]:
x=base_model.output
x=Conv2D(512, (8, 8), activation='relu')(x)
#x=MaxPooling2D(pool_size=(2, 2))(x)
x=Flatten()(x)
x=Dense(512,activation='relu')(x) #dense layer 3
x=Dropout(rate=0.25)(x)
preds=Dense(14,activation='softmax')(x) #final layer with softmax activation

model=Model(inputs=base_model.input, outputs=preds)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [30]:
for i,layer in enumerate(model.layers):
  print(i,layer.name)

0 input_1
1 block1_conv1
2 block1_conv2
3 block1_pool
4 block2_conv1
5 block2_conv2
6 block2_pool
7 block3_conv1
8 block3_conv2
9 block3_conv3
10 block3_pool
11 block4_conv1
12 block4_conv2
13 block4_conv3
14 block4_pool
15 block5_conv1
16 block5_conv2
17 block5_conv3
18 block5_pool
19 conv2d_1
20 flatten_1
21 dense_1
22 dropout_1
23 dense_2


In [0]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in model.layers[:19]:
    layer.trainable=False
for layer in model.layers[19:]:
    layer.trainable=True

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer=Adam(lr=0.001), 
              loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
es = EarlyStopping(monitor='val_loss', mode='min', min_delta=0, patience=15)

In [33]:
# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(x_train)


# fits the model on batches with real-time data augmentation:
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size, subset="training"),
                   steps_per_epoch=int(len(x_train)*0.8) / batch_size, epochs=epochs,
                   validation_data = datagen.flow(x_train, y_train, batch_size=batch_size, subset='validation'),
                   validation_steps = int(len(x_train)*0.2) / batch_size,
                   verbose=1, callbacks=[es])

model.save(model_name)

Instructions for updating:
Use tf.cast instead.
Epoch 1/1000
16/15 [==============================] - 22s 1s/step - loss: 5.3691 - acc: 0.1852 - val_loss: 3.4448 - val_acc: 0.2823
Epoch 2/1000
16/15 [==============================] - 8s 503ms/step - loss: 2.3239 - acc: 0.4315 - val_loss: 1.2887 - val_acc: 0.6210
Epoch 3/1000
16/15 [==============================] - 10s 611ms/step - loss: 1.1436 - acc: 0.6327 - val_loss: 0.9866 - val_acc: 0.7581
Epoch 4/1000
16/15 [==============================] - 10s 652ms/step - loss: 0.7610 - acc: 0.7731 - val_loss: 0.7557 - val_acc: 0.7339
Epoch 5/1000
16/15 [==============================] - 10s 643ms/step - loss: 0.6404 - acc: 0.7887 - val_loss: 0.7038 - val_acc: 0.7661
Epoch 6/1000
16/15 [==============================] - 10s 647ms/step - loss: 0.5375 - acc: 0.8125 - val_loss: 0.6507 - val_acc: 0.7742
Epoch 7/1000
16/15 [==============================] - 10s 647ms/step - loss: 0.5012 - acc: 0.8419 - val_loss: 0.6195 - val_acc: 0.7742
Epoch 8/100

In [34]:
predictions_test = model.predict(x_test, verbose=1)

data = pd.DataFrame()
data['Id'] = test_files
data['Category'] = predictions_test.argmax(axis=-1)


data.to_csv("envio.csv", index=False)

146/146 [==============================] - 4s 29ms/step


In [35]:
conversion = {
    'a': 0,
    'b': 1,
    'c': 2,
    'd': 3,
    'e': 4,
    'f': 5,
    'g': 6,
    'h': 7,
    'i': 8,
    'j': 9,
    'k': 10,
    'l': 11,
    'm': 12,
    'n': 13
}
classes = []
for t in test_files:
  class_ = t.split("T")[0]
  classes.append(conversion[class_])

from sklearn.metrics import accuracy_score
print(accuracy_score(classes, data['Category']))

0.8424657534246576
